# Handwritten Character Recognition: Using Machine Learning to Detect a 'W' in the demographic column

This notebook builds upon the image processing work in the document segmentation notebook. The code from that notebook has been consolidated into a file called 'segmentation.py' and will be imported for use in this notebook. This notebook will demonstrate a machine learning approach to recognizing handwritten characters, using a layered *neural network* that is trained on the publically available EMNIST letters dataset. The E in EMNIST stands for Extended, as it is an extended version of the MNIST dataset, which is the Modified National Institute of Standards and Technology (NIST) dataset of handwritten digits. Before you can train a machine learning model (or neural network) on the digit dataset you have to download it from the NIST website. Lucky for us, the Keras toolkit now includes functions that automate the download of the dataset the first time that you use it. Be forewarned that this code will take a long time to execute the first time, depending upon your network speed.

This notebook is partially based on the [Keras MNIST tutorial](https://colab.research.google.com/github/AviatorMoser/keras-mnist-tutorial/blob/master/MNIST%20in%20Keras.ipynb) written by Daniel Moser (UT Southwestern Medical Center). We modify their approach to focus on letters exclusively and explore training a NN and Convolutional NN (CNN) to specialize in recognizing the W character alone.

## Install Dependencies
The first step is to install some necessary Python libraries for machine learning. We are using the Google suite of tools, which include *TensorFlow* and *Keras*, which is a somewhat simpler API that wraps the base functions provided by TensorFlow. Please Note: There is a strange situation with Keras version 3 that requires it to be installed (or reinstalled) after you install tensorflow. (TensorFlow tries to install it's own earlier version of Keras.)

Be sure to restart your kernel after these tools have installed.

In [3]:
%pip install --upgrade tensorflow numpy tensorflow-datasets
%pip install --upgrade keras=="3"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Loading the EMNIST image data into a TensorFlow Dataset

The next step is to setup the tensorflow training dataset that contains the digits that we want to use to train a model. EMNIST includes thousands of grayscale images that are 28x28 pixels in size with 256 levels of gray. The images are numerically labelled with the letter represented in the image. For instance, a 'W' is labelled with the number 23. The following code uses the TensorFlow Datasets module to load our chosen dataset. It will download the data the first time that you run it, which can take some time. Notice that we have specified some options to the `load()` function, saying that we desire only letters. It also specifies that we want the dataset split into  both training data and testing data. The testing data will be used to measure the accuracy of our model after training. We also load information about the dataset into a separate Python dictionary.

First we print out the dataset info object, then we display a grid of the first 9 images in the dataset, so that you can see what the images look like.

In [4]:
import numpy as np                   # advanced math library
import matplotlib.pyplot as plt      # MATLAB like plotting routines

#from tensorflow.keras.utils import to_categorical              # NumPy related tools

# Data should be located in a folder called EMNIST
import tensorflow_datasets as tfds
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/letters',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print(ds_info)

plt.figure(figsize=(8, 8))
for i, (img, lbl) in enumerate(ds_train.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.title(str(int(lbl)))
    plt.axis("off")

2024-04-13 13:01:37.275937: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 13:01:37.276232: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 13:01:37.278760: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 13:01:37.306870: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 13:01:37.815254: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

tfds.core.DatasetInfo(
    name='emnist',
    full_name='emnist/letters/3.0.0',
    description="""
    The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19 and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset.
    
    Note: Like the original EMNIST data, images provided here are inverted horizontally and rotated 90 anti-clockwise. You can use `tf.transpose` within `ds.map` to convert the images to a human-friendlier format.
    """,
    config_description="""
    EMNIST Letters
    """,
    homepage='https://www.nist.gov/itl/products-and-services/emnist-dataset',
    data_dir='/home/jansen/tensorflow_datasets/emnist/letters/3.0.0',
    file_format=tfrecord,
    download_size=535.73 MiB,
    dataset_size=44.14 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=37),
    }),
    supervis

2024-04-13 13:01:38.143004: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-13 13:01:38.143266: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


<IPython.core.display.Javascript object>

2024-04-13 13:01:38.403440: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-04-13 13:01:38.410296: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


You may notice that the letters look pretty strange. They are in fact inverted horizontally and rotated 90 degrees anti-clockwise. This is a legacy of the original EMNIST dataset and is maintained in the tensorflow version for consistency. You can read this explanation in the dataset information we printed above. There are also instructions there for how to make the images match our expectations and we will follow those in steps below.

**Please Note:**  The number of classes (labels) for the images is listed as 37 in the dataset information dictionary. This is in fact a typo in the TensorFlow dataset, as of 3.0 version still there. The same typo occurs in the original EMNIST paper, where the number of classes in the "letters" subset is reported as both 37 AND 26 in different places. I discovered this discrepancy towards the end of my analysis. We can simply treat the number of classes as 26 and ignore the incorrect 37.

## Census Data

Let's load and preview some census demographic code images for comparison. Remember that we are training a model to recognize the census W's based entirely on these EMNIST examples. The two data sources do not resemble each other much yet, but we can take steps to bring them closer together.

In [5]:
# First we load the segmentation code that we developed in that notebook
with open("segmentation.py") as f:
    code = f.read()
exec(code)

page = "pages/43290879-California-101393-0002.jpeg"
cells = list(generateCellImages(page))
plt.figure(figsize=(8, 8))
for i, image in enumerate(cells[:9]):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(str(i))
    plt.axis("off")


<IPython.core.display.Javascript object>

If you compare the Census 'W' images to the EMNIST letters, you should notice a few things:

* The EMNIST images are 28 by 28 pixels square, whereas the Census cell images are taller and wider.
* The EMNIST images are rotated and flipped, whereas the Census cell images are oriented normally.
* The Census images contain some extraneous ink, usually a horizontal rule that marks the botton of the demographic cell. This noisy line is sometimes faint and sometimes heavy. The EMNIST image consist of clear and crisp letters with no extraneous ink.

To make the machine learning approach work best, we will adjust our data to correct these differences. First let's pick the best 28 by 28 pixel square from the Census cells. We define a function *crop_cell()* to do this work for us. We ideally want to center the square on the W character, or whatever is written within the cell. To approximate that we'll find the "center of ink" within the cell and make that point, call the moment, the center of our square. If the center of mass is too close to an edge we'll adjust the crop so that the square aligns with that edge.

In [6]:
def crop_cell(cell_img, debug=False):
    M = cv2.moments(cell_img)
    X = int(M["m10"] / M["m00"])
    Y = int(M["m01"] / M["m00"])
    x1 = min(max(0, X-13), cell_img.shape[0] - 28)
    y1 = min(max(0, Y-13), cell_img.shape[1] - 28)
    cell = cell_img[x1:x1+28, y1:y1+28]
    cv2.rectangle(cell_img, (x1, y1, 28, 28), RED, thickness=1)
    view(cell_img, "crop region")    
    return cell

page = "pages/43290879-California-101393-0002.jpeg"
cells = list(generateCellImages(page))
for image in cells[:3]:
    crop_cell(image, debug=True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>



## Transpose ENMIST Letters to a Readable Orientation

Now that we have a 28x28 square census image, let's make sure that we unrotate and unflip the EMNIST images so that they look like the normal readable letters from the Census. In order to do that we are going to use a TensorFlow *map()* operation to apply the same function, in a streaming fashion as needed, to every image in the training and testing sets. The function we will apply to every image is the TensorFlow *transpose()* function, which is the one recommended in the dataset notes. You can see that our preview grid is now readable.

In [7]:
import tensorflow as tf

import tensorflow_datasets as tfds
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/letters',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_train_readable = ds_train.map(
  lambda img, label: (tf.transpose(img, perm=[1,0,2]), label),
  num_parallel_calls=tf.data.AUTOTUNE, 
  deterministic=True)

plt.figure(figsize=(8, 8))
for i, (img, lbl) in enumerate(ds_train_readable.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.title(str(int(lbl)))
    plt.axis("off")

<IPython.core.display.Javascript object>

2024-04-13 13:01:40.028166: E tensorflow/core/util/util.cc:131] oneDNN supports DT_UINT8 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
2024-04-13 13:01:40.090574: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-04-13 13:01:40.096768: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Adding Noise to Match Census Data

Okay, we're almost ready. Now we have to account for the additional noisy ink in the Census images. This ink is pretty hard to remove from around each character in the Census images. Instead of removing ink from the Census images, we are going to try adding similar ink to the training and testing data from EMNIST. This seems counterintuitive to us because we associate letters with concepts and their particular shapes. To a machine learning model these letters are simply patterns of pixels. We'll train a model to recognize letters with noisy lines added to them and this will be easier that removing those lines using computer vision techniques.

We use the same *map()* operation to add noise to the EMNIST images. One thing to note here is that while we are dealing with 28x28 pixel images, the TensorFlow datasets are provided as matrices with three dimensions. This is because the images are provided as batches, with multiple images provided in one larger matrix. Hence the functions we are mapping below have to deal in three dimensional inputs and outputs.

In [8]:

# This function is used to add two matrices together without exceeding the pixel value of 255.
def add_to_255(a, b):
    """Return a+b to a max value of 255"""
    if a + b > 255:
        return 255
    else:
        return a + b
v_add_to_255 = np.vectorize(add_to_255)  # Here we create a "vectorized" form of our simple function above.

def noisy_line(img, label):
    """Adds background noise and a vertical line on the right edge to match data"""
    rng = np.random.default_rng()  # This function uses a random number generator, created here.
    line = np.zeros((28,28,1), dtype=np.uint8)  # Our line will be defined in a 28x28 matrix, initial set to zeroes.
    
    # We pick a random number c, from 20 to 25, as the center position of our horizontal line.
    c = rng.integers(20, high=25, size=1)[0]  # randomized position of line
    a = rng.integers(0, high=100, size=1)[0]  # randomized transparency of line

    # We set the center of the line to a max of 200 and adjacent rows to a max of 100.
    line[c,:] = (100 - a)
    line[c+1,:] = (200 - 2 * a)
    line[c+2,:] = (100 - a)
    noise = rng.integers(0, high=(100-a),size=(3,28,1))
    line[c:c+3,:] = line[c:c+3,:] - noise
    img = v_add_to_255(img, line)
    return img, label

# Some examples of the noise we will add to EMNIST
z = np.zeros((28,28, 1), dtype=np.uint8)
view(noisy_line(z, 0)[0])  # The views show normalized luminence, i.e. they treat the highest value found as white.
view(noisy_line(z, 1)[0])  # This means that our transparency value has no impact in these previews of the noisy lines.
view(noisy_line(z, 2)[0])

# Some examples of EMNIST letters with noise added.
plt.figure(figsize=(8, 8))
for i, (image, label) in enumerate(ds_train_readable.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    #img_numpy = image.numpy()[..., None]  # Adding a dimension to match TF dataset API
    img_numpy = image.numpy()  # Adding a dimension to match TF dataset API
    img, lbl = noisy_line(img_numpy, label.numpy())
    plt.imshow(img)
    plt.title(str(int(lbl)))
    plt.axis("off")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2024-04-13 13:01:40.253670: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-04-13 13:01:40.260103: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


These images with noisy horizontal lines are a better match for the Census data. We expect that any model trained to recognize these letters will also recognize 'W' characters from the Census data.

## Applying Functions (Mapping) and Normalizing to Floating Point Values

We have shown some examples of the transformations that we need, both to the Census images and to the EMNIST letters training and testing data. Now we will apply those tranformation to the EMNIST training and testing data. Each transformation is applied in a series of steps using the *map()* function.

This code is a bit hard to read. When training our model the TensorFlow API will pull data from the training dataset. As each training image is prepared for use, it will apply our mapped functions. This code tells the TensorFlow API which functions are mapped onto the dataset and in what order they are to be applied. The testing dataset is prepared similarly.

### Further Details

For those interested in more detail. The TensorFlow API likes to precache and batch training data for performance reasons. The API user has to defined the size of these batches. Batches have the effect of reshaping the data, adding a dimension to the tensor arrays for the batch index. So instead of a 28x28 image and a label, after batching you have a 128x28x28 array of data. This can be pretty confusing to those new to TensorFlow and makes it harder to interpret training error messages about the shape and dimensionality of the training data.

Because our noisy line function converts data into a numpy array and then back into a Tensor, the dataset looses track of the training element shape. Our set_shapes function will reinform the API of our data's shape. By printing out the element shape information at various points, we can see how the shape and dimensionality of our training data changes as functions are mapped.

In [9]:
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'emnist/letters',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print("initial element spec", ds_train.element_spec)

ds_train_readable = ds_train.map(
  lambda img, label: (tf.transpose(img, perm=[1,0,2]), label),
  num_parallel_calls=tf.data.AUTOTUNE, 
  deterministic=True)

# Mapping the noisy line function to the dataset..
ds_train_readable_noisy = ds_train_readable.map(
  lambda img, label: tf.numpy_function(func=noisy_line, inp=[img, label], Tout=(tf.uint8, tf.int64)),
  num_parallel_calls=tf.data.AUTOTUNE, 
  deterministic=False)

# Function to convert image pixel values to floating point between 0 and 1.
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

# Mapping the normalize function to the dataset
ds_train_readable_noisy_float = ds_train_readable_noisy.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

def set_shapes(image, label):
  image.set_shape([28, 28, 1])
  label.set_shape([])
  return image, label

# Assert the correct tensor shapes
ds_train_final = ds_train_readable_noisy_float.map(set_shapes, num_parallel_calls=tf.data.AUTOTUNE)

# Some additional dataset setup steps
ds_train_final = ds_train_final.cache()
ds_train_final = ds_train_final.batch(128)  # This changes the shape of the data, so call it after all mapped functions..
ds_train_final = ds_train_final.prefetch(tf.data.AUTOTUNE)
print("final element_spec", ds_train_final.element_spec)

# Then we need to apply the same functions and settings to the test dataset
ds_test_readable = ds_test.map(
  lambda img, label: (tf.transpose(img, perm=[1,0,2]), label),
  num_parallel_calls=tf.data.AUTOTUNE, 
  deterministic=True)

ds_test_readable_noisy = ds_test_readable.map(
  lambda img, label: tf.numpy_function(func=noisy_line, inp=[img, label], Tout=(tf.uint8, tf.int64)),
  num_parallel_calls=tf.data.AUTOTUNE, 
  deterministic=False)
ds_test_readable_noisy_float = ds_test_readable_noisy.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test_final = ds_test_readable_noisy_float.map(set_shapes, num_parallel_calls=tf.data.AUTOTUNE)
ds_test_final = ds_test_final.batch(128)
ds_test_final = ds_test_final.cache()
ds_test_final = ds_test_final.prefetch(tf.data.AUTOTUNE)
print("final test element_spec", ds_test_final.element_spec)

initial element spec (TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))
final element_spec (TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))
final test element_spec (TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))


Above you can see that the final element spec is 4-dimensional, including the dimension used for the batch index. The final datatype has changed to floating point, i.e. values between zero and one. Even the labels in the 2nd part of the element spec tuple have changed shap to reflect the batching dimension.

## Defining and Training a Machine Learning Model

We are going to define the most straightforward machine learning model first and customize it for our parameters and the output we want. This model is defined as a sequence that takes in a 28 by 28 tensor array. The input is first flattened into a 784 long tensor array of one dimension (28 times 28). Then each of these *input features* is connected to each of the units in the subsequent neural network layer. The dense NN layer stores weights associated with each of these connections and those weights are used to determine the value each unit will pass to the next layer in the model. These weights are adjusted during training. An activation formular (relu) is supplied to this layer. The default activation formation is "linear", which means that unit output pretty directly reflect input and weights. Our activation formula of [ReLU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/ReLU) means that unit output never exceeds a certain maximum value and that any unit output below a certain threshold is attenuated, or lessened. This privileges the output of units that are strongly weighted and also receiving input.

Each unit in the first dense layer is subsequently connected to all of the 27 units in the last dense layer. The activation there is *softmax*, not ReLU. [Softmax](https://www.tensorflow.org/api_docs/python/tf/keras/activations/softmax) is an activation that converts the output vector or 27 possibilities into a probability distribution, where all of the output values sum to 1. That makes it easier to compare the confidence of a model's predictions.

After you defined the neural network structure and activations in the first step, you compile that model, adding an optimizer, a loss function, and a metrics function. The loss function compare each prediction to the expected result in the training data. The model will be trained in steps to minimize this loss. Our loss function is categorical, which means that the model is judged on how well it chooses a category (a letter) based on the input it receives. The summary function prints a nice display of the model we have constructed.

In [10]:
model = tf.keras.models.Sequential([
  tf.keras.Input(shape=(28,28), name='input'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(27, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 128)               │    100,480 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 27)                │      3,483 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 103,963 (406.11 KB)

 Trainable params: 103,963 (406.11 KB)

 Non-trainable params: 0 (0.00 B)

In the summary above you can see how many trainable parameters there are within the model we have defined. Each trainable parameter is the weight of a connection between one layer and the next. So 784 inputs are each connected to all 128 units (or neurons) in the dense layer, for 784 x 128 weights or 100,480 parameters. Then in the next layer you have 128 outputs connected to 27 units, for 128 x 27 weights, or 3,483 parameters. The total therefore is 103,963 trainable connections between inputs and neuron units.

Next, the *fit()* function performs the training, fitting the model to the training data. This can take a few minutes, depending upon the speed of your computer and available memory. The training is performed in 15 *epochs* and each epoch is a complete pass through the training data. As training is performed you will see metrics that show how accurate the model is at predicting labels in the test dataset.

In [11]:

model.fit(
    ds_train_final,
    epochs=15,
    validation_data=ds_test_final,
)

Epoch 1/15


/home/jansen/.local/share/virtualenvs/census-form-image-processing-VTyG8Y6d/lib/python3.10/site-packages/keras/src/backend/tensorflow/nn.py:568: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


694/694 ━━━━━━━━━━━━━━━━━━━━ 28s 40ms/step - loss: 1.4904 - sparse_categorical_accuracy: 0.5867 - val_loss: 0.7721 - val_sparse_categorical_accuracy: 0.7753
Epoch 2/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6747 - sparse_categorical_accuracy: 0.8047 - val_loss: 0.6026 - val_sparse_categorical_accuracy: 0.8224
Epoch 3/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5229 - sparse_categorical_accuracy: 0.8461 - val_loss: 0.5362 - val_sparse_categorical_accuracy: 0.8396
Epoch 4/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4499 - sparse_categorical_accuracy: 0.8657 - val_loss: 0.5007 - val_sparse_categorical_accuracy: 0.8484
Epoch 5/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4033 - sparse_categorical_accuracy: 0.8781 - val_loss: 0.4775 - val_sparse_categorical_accuracy: 0.8528
Epoch 6/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3701 - sparse_categorical_accuracy: 0.8858 - val_loss: 0.4619 - val_sparse_categorical_accuracy: 0.8577
Epoch 7/15
69

Superb! Look at that machine learn! In my training I am seeing a accuracy of 92.5 percent. You can see that in each epoch the loss is decreasing and the accuracy is improving as the weights (or trainable parameters) are adjusted.

You might wonder why the progress bars above are counting up to 694.. Can you guess? If each epoch is one pass through all the training data and each epoch is divided into 694 steps, then what is each step?

The answer is that each epoch is divided into the batches of 128 we defined in the training data.

The number of training examples in EMNIST letters is 88,800. This divides up into 694 batches of 128 examples, with the last batch being somewhat smaller (the remainder).

## Predicting a 'W' in the Census Images

Our next step is to attempt to use the model to predict if a Census image is a 'W' or not. Let's try that with the model we've just trained.

The code below is an improvement on some of the *run()* functions we have used before in the image segmentation notebook. This run function takes a folder path, a page range, and a model. Since the census images we downloaded are named with a series of page numbers encoded after a final hyphen, such as "43290879-California-101393-0014.jpeg", we can use a regular expression to detect if a page is within the range of integers desired. If it is not, then we simply skip it. The *glob()* function will return files within the desired folder path and the *sort()* function will ensure that those files are returned in the right page order.

After we have a page image file we read in the image with OpenCV *imread()*. The image is passed to the *extract()* function from the `segmentation.py` file, which returns the properly positioned template for the main part of the census form as well as a deskewed and adjusted image we can use to extract cell images. We then use the template to calculate a list of rectangles or boxes that represent demographic cells from the census page (`demographic_cells`).

For each demographic cell, we crop a little image out of the page image (`cell_img`). We crop the cell image down to a 28x28 pixel square, using the *moments()* function to grab the most ink possible without exceeding the edges of the little cell image. The 28x28 square is then normalized, just like our training data, into a tensor of floating point values. This tensor is expanded with an empty third dimension, just so that it matches our training data, which was three dimensions after it was batched. Having done this last step, we can finally call the *predict()* function and see what the model says about the cell.

The result is a tensor array of 27 floating point values that represent the probability distribution among the 27 possible categories, i.e. letters. You can print out the full result tensor if that is interesting to you. Our code selects the index with the maximum probability (highest value). For each page we plot all of the demographic cells in a 6 by 5 grid, with a view of each 28x28 cropped cell image and caption that includes the class (integer label), the letter (looked up in a list), and the probability value of the prediction.

Have a good look at the resulting plots of images and their predicted categories. What sort of image features impact predictions and their confidence? Are all of the 'W's predicted accurately? What other letters are often predicted instead of 'W' and why?

In [12]:
import glob
import re
import cv2

lookup = list("abcdefghijklmnopqrstuvwxyz".upper())

def run(path, page_range=(2, 16), model=None, debug=False):
    for f in sorted(glob.glob(f'{path}/*')):
        pagestr = re.search(r'-(\d+).jpeg', f).group(1)
        if int(pagestr) not in range(page_range[0], page_range[1]):
            continue
        plt.figure(figsize=(16, 16))
        print(f)
        image = cv2.imread(f)
        (adjusted_img, v_lines, h_lines) = extract(image, f, debug=debug)
        grayimage = cv2.cvtColor(adjusted_img,cv2.COLOR_BGR2GRAY)
        grayimage = 255 - grayimage
        demo_h_offset = v_lines[11]  # the demographic column starts the the 12th vertical line
        demo_width = v_lines[12] - demo_h_offset  # width calculation
        # Open CV rectangles calculation for each demographic cell
        demographic_cells = \
          [ (demo_h_offset, h_lines[i], demo_width, int(h_lines[i+1]-h_lines[i])) for i in range(3, len(h_lines)-1)]
        for i in range(3, len(h_lines)-1):
            ax = plt.subplot(5, 6, i - 2)
            plt.axis('off')
            cell_img = grayimage[h_lines[i]+5:h_lines[i+1]+5, v_lines[11]:v_lines[12]]
            M = cv2.moments(cell_img)
            X = int(M["m10"] / M["m00"])
            Y = int(M["m01"] / M["m00"])
            x1 = min(max(0, X-13), cell_img.shape[0] - 28)
            y1 = min(max(0, Y-13), cell_img.shape[1] - 28)
            cell = cell_img[x1:x1+28, y1:y1+28]
            tensor, noop = normalize_img(cell, 0)
            result = model.predict(np.expand_dims(tensor, 0), verbose=0)
            classes_x=np.argmax(result,axis=1)
            plt.imshow(cell)
            plt.title(f'{lookup[int(classes_x[0])-1]} ({result[0][int(classes_x[0])]:.2f}) W ({result[0][23]:.2f})')
        plt.show()


run('pages', page_range=(14,17), model=model)

<IPython.core.display.Javascript object>

pages/43290879-California-101393-0014.jpeg


<IPython.core.display.Javascript object>

pages/43290879-California-101393-0015.jpeg


<IPython.core.display.Javascript object>

pages/43290879-California-101393-0016.jpeg


## Using a Convolutional Neural Network (CNN)

In [13]:
conv_model = tf.keras.models.Sequential(
  [
    tf.keras.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(27, activation="softmax"),
  ] 
)

conv_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

conv_model.summary()

conv_model.fit(
    ds_train_final,
    epochs=15,
    validation_data=ds_test_final,
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)        │        320 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)        │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)        │     18,496 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)          │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout (Dropout)               │ (None, 1600)              │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_2 (Dense)                 │ (None, 27)                │     43,227 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 62,043 (242.36 KB)

 Trainable params: 62,043 (242.36 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 1.4049 - sparse_categorical_accuracy: 0.5874 - val_loss: 0.4904 - val_sparse_categorical_accuracy: 0.8477
Epoch 2/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.5090 - sparse_categorical_accuracy: 0.8400 - val_loss: 0.3928 - val_sparse_categorical_accuracy: 0.8759
Epoch 3/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.4158 - sparse_categorical_accuracy: 0.8688 - val_loss: 0.3462 - val_sparse_categorical_accuracy: 0.8897
Epoch 4/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.3753 - sparse_categorical_accuracy: 0.8794 - val_loss: 0.3277 - val_sparse_categorical_accuracy: 0.8950
Epoch 5/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.3453 - sparse_categorical_accuracy: 0.8881 - val_loss: 0.3084 - val_sparse_categorical_accuracy: 0.9037
Epoch 6/15
694/694 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.3221 - sparse_categorical_accuracy: 0.8932 - val_loss: 0.2916 - val_sparse_categorical_accuracy: 0.9061
Epoc

In [14]:
run('pages', page_range=(14,17), model=conv_model)

<IPython.core.display.Javascript object>

pages/43290879-California-101393-0014.jpeg


<IPython.core.display.Javascript object>

pages/43290879-California-101393-0015.jpeg


<IPython.core.display.Javascript object>

pages/43290879-California-101393-0016.jpeg


## Looking at Predictions

In the results above you can probably see several instances where a 'W' image was predicted to be some other letter. There may be some remaining way to improve the accuracy of the model's predictions, such as a more true to life "noisy line" rendering, or a more sophisticated model. However, any model is going to be less than 100% on real world data. As you can see, many census demographic cells contain what look like meaningless scribbles without the context of the page. Recorders would often make notes in the unused portion of the main table of a page, for instance.

Given that any model will be imperfect, what can we do to make these results more useful? Stop and think about that question and remember that our goal in using this 'W' detector is to save time on manual transcription tasks. If we can quickly review and skip pages that do not contain the target demographic for transcription, then we will save considerable time and mental load.

## Making Partial Predictions More Useful

There are several steps we can take that will make our predictions more useful in filtering data for manual transcription.

1. Filter out all of the known 'W' characters. There is no need to show people images of demographic cells that we are sure are 'W' characters.

1. Filter out all the cells where 'W' was a significant probability. Even though the model may predict another character is more likely than a 'W', we often see that a 'W' is in second place, in terms of probability. A quick review of these situations indicates that in most cases, with respect to our census demographic data, these second place 'W' predictions are correct. So we can probably filter out the cells where 'W' is the second most predicted character.

1. Our task is really to collect page references for further analysis by a human being, who will look at the whole page. If we can help a person focus on the few cells that may or may not contain a targetted demographic code ("Jap" is the demographic code for Japanese, in our case), then a person can either say yes or no to saving that page reference for further analysis. In other words, even though we may have up to 30 cell images to review, we only need one answer for the entire page, which is "save" or "ignore".

Let's see if we can implement something like this in our *run()* function. First we will filter out all the known 'W' cells and cells where 'W' is a significant second place prediction, say greater than 15%.

In [ ]:
%matplotlib notebook
import glob
import re
import cv2
import IPython
import ipywidgets as widgets
from IPython.display import display
from jupyter_ui_poll import ui_events
import time

lookup = list("abcdefghijklmnopqrstuvwxyz".upper())
blank = np.zeros((28,28, 1), dtype=np.uint8)
waiting_for_click = True
current_page = ""
saved_pages = []

def on_save_button(x):
    global waiting_for_click
    global saved_pages
    print(f'saved')
    saved_pages.append(current_page)
    waiting_for_click = False

def on_ignore_button(x):
    global waiting_for_click
    waiting_for_click = False

def run(path, page_range=(2, 16), model=None, debug=False):
    global current_page
    global waiting_for_click
    plt.ioff()
    out = widgets.Output(layout={'border': '1px solid black'})
    display(out)
    #with out:
    for f in sorted(glob.glob(f'{path}/*')):
        pagestr = re.search(r'-(\d+).jpeg', f).group(1)
        if int(pagestr) not in range(page_range[0], page_range[1]):
            continue
        fig = plt.figure(figsize=(30, 10))
        current_page = f
        out.append_stdout(f)
        image = cv2.imread(f)
        (adjusted_img, v_lines, h_lines) = extract(image, f, debug=debug)
        grayimage = cv2.cvtColor(adjusted_img,cv2.COLOR_BGR2GRAY)
        grayimage = 255 - grayimage
        demo_h_offset = v_lines[11]  # the demographic column starts the the 12th vertical line
        demo_width = v_lines[12] - demo_h_offset  # width calculation
        # Open CV rectangles calculation for each demographic cell
        demographic_cells = \
        [ (demo_h_offset, h_lines[i], demo_width, int(h_lines[i+1]-h_lines[i])) for i in range(3, len(h_lines)-1)]
        for i in range(3, len(h_lines)-1):
            ax = plt.subplot(3, 10, i - 2)
            plt.axis('off')
            cell_img = grayimage[h_lines[i]+5:h_lines[i+1]+5, v_lines[11]:v_lines[12]]
            M = cv2.moments(cell_img)
            X = int(M["m10"] / M["m00"])
            Y = int(M["m01"] / M["m00"])
            x1 = min(max(0, X-13), cell_img.shape[0] - 28)
            y1 = min(max(0, Y-13), cell_img.shape[1] - 28)
            cell = cell_img[x1:x1+28, y1:y1+28]
            tensor, noop = normalize_img(cell, 0)
            result = model.predict(np.expand_dims(tensor, 0), verbose=0)
            classes_x=np.argmax(result,axis=1)
            prediction = int(classes_x[0])
            if prediction == 23 or result[0][23] >= .15:
                plt.imshow(blank)
            else:
                ax.set(facecolor="orange")
                plt.imshow(cell)
            plt.title(f'{lookup[int(classes_x[0])-1]} ({result[0][int(classes_x[0])]:.2f}) W ({result[0][23]:.2f})')
        display(fig, out)
        # prompt with buttons
        waiting_for_click = True
        save_button = widgets.Button(description="Save")
        save_button.on_click(on_save_button)
        ignore_button = widgets.Button(description="Ignore")
        ignore_button.on_click(on_ignore_button)
        display(save_button, out)
        display(ignore_button, out)
        with ui_events() as poll:
            while waiting_for_click == True:
                #waiting for user input
                poll(10) # poll queued UI events including button
                time.sleep(10)
                pass
        out.clear_output()

run('pages', page_range=(14,17), model=model)